In [11]:
import sys
sys.path.append('../')
import functions
exec(open('./functions.py', encoding="utf8").read())

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\chiso\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\chiso\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\chiso\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
... OK (Got response in 0.54 seconds)
Writing emoji data to C:\Users\chiso\.demoji\codes.json ...
... OK


In [11]:
# load the preprocessed doc2vec data
data = pickle.load(open('data_n_models/eng_df_represented_doc2vec.pkl', 'rb'))
data.head()

,compound,0,1,2,3,4
0,0.0000,-0.015905,0.191896,0.311980,-0.082201,-0.054656
1,-0.9060,0.094732,-0.196510,0.232367,-0.184807,-0.250304
2,0.6249,-0.028408,0.077553,0.232172,-0.400177,0.024043
3,-0.4767,-0.025499,-0.010442,-0.289881,-0.029361,-0.232676
4,0.0000,-0.092368,0.046109,0.195366,-0.112181,-0.068188


In [12]:
# change compound from continous to categorical variable
data["compound"] = np.where(data["compound"] > 0, 1, data["compound"])
data["compound"] = np.where(data["compound"] < 0, -1, data["compound"])

In [13]:
data.head()

,compound,0,1,2,3,4
0,0.0,-0.015905,0.191896,0.311980,-0.082201,-0.054656
1,-1.0,0.094732,-0.196510,0.232367,-0.184807,-0.250304
2,1.0,-0.028408,0.077553,0.232172,-0.400177,0.024043
3,-1.0,-0.025499,-0.010442,-0.289881,-0.029361,-0.232676
4,0.0,-0.092368,0.046109,0.195366,-0.112181,-0.068188


In [14]:
# assing the X and y variables
X = data.drop("compound", axis=1)
y = data["compound"]

In [15]:
# check X
X.head()


,0,1,2,3,4
0,-0.015905,0.191896,0.311980,-0.082201,-0.054656
1,0.094732,-0.196510,0.232367,-0.184807,-0.250304
2,-0.028408,0.077553,0.232172,-0.400177,0.024043
3,-0.025499,-0.010442,-0.289881,-0.029361,-0.232676
4,-0.092368,0.046109,0.195366,-0.112181,-0.068188


In [16]:
# check y
y.head()

0    0.0
1   -1.0
2    1.0
3   -1.0
4    0.0
Name: compound, dtype: float64

In [17]:
# split  to test and train set - use 80% for training and 20% for test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= 0.2, random_state=42, stratify= y)

In [18]:
# train the model
clf = LogisticRegression(multi_class="multinomial", solver="lbfgs")
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

        -1.0       0.47      0.26      0.33      4946
         0.0       0.32      0.06      0.11      7532
         1.0       0.45      0.84      0.58      9520

    accuracy                           0.44     21998
   macro avg       0.41      0.39      0.34     21998
weighted avg       0.41      0.44      0.36     21998



In [19]:
# create a balanced data set, train and test our model on it
shuffled_df = data.sample(frac=1, random_state= 42)
print(shuffled_df["compound"].value_counts())
shuffled_df.head()


 1.0    47597
 0.0    37658
-1.0    24732
Name: compound, dtype: int64


,compound,0,1,2,3,4
39370,-1.0,-0.029322,0.249719,0.286637,0.063639,-0.072456
90544,0.0,-0.076301,0.089977,-0.088174,-0.025999,-0.029968
28550,-1.0,0.081735,0.216456,0.033149,0.000074,0.099212
86146,-1.0,0.006570,0.087801,0.419855,-0.216380,-0.064295
41622,0.0,-0.023165,0.192196,0.151058,-0.155872,0.028333


In [20]:
# put all the different compounds in a separate data set
pos = shuffled_df.loc[shuffled_df["compound"] == 1]
neu = shuffled_df.loc[shuffled_df["compound"] == 0]
neg = shuffled_df.loc[shuffled_df["compound"] == -1]

In [21]:
# balance them all to 24732, the smallest sample
neu = neu.sample(n=24732, random_state= 42)
pos = pos.sample(n=24732, random_state= 42)

In [22]:
# concatenate all dataframes
balanced_df = pd.concat([pos, neu, neg])
balanced_df.shape

(74196, 6)

In [23]:
balanced_df.head()

,compound,0,1,2,3,4
108614,1.0,0.005523,0.069174,0.066438,-0.061880,0.114920
47673,1.0,-0.060819,0.144103,0.176268,0.013342,-0.040103
98694,1.0,-0.201348,0.186996,0.256563,-0.069427,-0.008400
90420,1.0,-0.110152,0.164517,0.212028,-0.356572,-0.024280
64814,1.0,-0.016312,0.053144,0.081005,-0.140801,0.024318


In [24]:
# assing the X and y variables
X_balanced = balanced_df.drop("compound", axis=1)
y_balanced = balanced_df["compound"]

In [25]:
# split  to test and train set - use 80% for training and 20% for test
X_trainb, X_testb, y_trainb, y_testb = train_test_split(X_balanced, y_balanced, test_size= 0.2, random_state=42, stratify= y_balanced)

In [26]:
# train the model
clfb = LogisticRegression(multi_class="multinomial", solver="lbfgs")
clfb.fit(X_trainb, y_trainb)
y_predb = clf.predict(X_testb)
print(classification_report(y_testb, y_predb))

              precision    recall  f1-score   support

        -1.0       0.62      0.26      0.37      4946
         0.0       0.32      0.07      0.11      4947
         1.0       0.36      0.84      0.50      4947

    accuracy                           0.39     14840
   macro avg       0.43      0.39      0.33     14840
weighted avg       0.43      0.39      0.33     14840



## KNN


In [52]:
grid_params = {
    'n_neighbors': [10,20,30,40,50,60,70,80,90,100, 110, 120, 130, 140, 150, 160, 170, 180, 190, 200],
    'weights': ['uniform', 'distance'],
    'metric': ['euclidean', 'manhattan']
}

gs = GridSearchCV(KNeighborsClassifier(),
    grid_params,
    verbose = 1,
    cv = 5,
    n_jobs = -1)

gs_results = gs.fit(X_train, y_train)

Fitting 5 folds for each of 80 candidates, totalling 400 fits


In [53]:
print(gs_results.best_score_)
print(gs_results.best_estimator_)
print(gs_results.best_params_)

0.4989145838387472
KNeighborsClassifier(metric='euclidean', n_neighbors=180, weights='distance')
{'metric': 'euclidean', 'n_neighbors': 180, 'weights': 'distance'}


In [61]:
knn = KNeighborsClassifier(n_neighbors=180, metric = 'euclidean', weights='distance')
knn.fit(X_train, y_train)
y_pred_knn = knn.predict(X_test)

# Confusion Matrix
pd.crosstab(y_pred_knn,y_test)

compound,-1.0,0.0,1.0
row_0,,,
-1.0,1170,410,705
0.0,1643,3909,2902
1.0,2133,3213,5913


In [62]:
print(classification_report(y_test, y_pred_knn ))

              precision    recall  f1-score   support

        -1.0       0.51      0.24      0.32      4946
         0.0       0.46      0.52      0.49      7532
         1.0       0.53      0.62      0.57      9520

    accuracy                           0.50     21998
   macro avg       0.50      0.46      0.46     21998
weighted avg       0.50      0.50      0.49     21998



In [56]:
# balanced data for KNN
gs_results_bal = gs.fit(X_trainb, y_trainb)
print(gs_results_bal.best_score_)
print(gs_results_bal.best_estimator_)
print(gs_results_bal.best_params_)

Fitting 5 folds for each of 80 candidates, totalling 400 fits
0.4877687399767628
KNeighborsClassifier(metric='euclidean', n_neighbors=110, weights='distance')
{'metric': 'euclidean', 'n_neighbors': 110, 'weights': 'distance'}


In [59]:
knn_bal = KNeighborsClassifier(n_neighbors=110, metric = 'euclidean', weights='distance')
knn_bal.fit(X_trainb, y_trainb)
y_pred_knn_bal = knn_bal.predict(X_testb)

# Confusion Matrix
pd.crosstab(y_pred_knn_bal,y_testb)

compound,-1.0,0.0,1.0
row_0,,,
-1.0,2401,1039,1123
0.0,1641,2862,1945
1.0,904,1046,1879


In [60]:
print(classification_report(y_testb, y_pred_knn_bal ))

              precision    recall  f1-score   support

        -1.0       0.53      0.49      0.50      4946
         0.0       0.44      0.58      0.50      4947
         1.0       0.49      0.38      0.43      4947

    accuracy                           0.48     14840
   macro avg       0.49      0.48      0.48     14840
weighted avg       0.49      0.48      0.48     14840



# Train the external data set and test it on our data using RandomForestClassifer.
#### eng_df_represented_doc2vec100.pkl & eng_ext_twt_doc2vec100.pkl are used for this purpose, the first our data and the second external data.

In [17]:
own_data = pickle.load(open('data_n_models/eng_df_represented_doc2vec100.pkl', 'rb'))
own_data.head()

,compound,0,1,2,3,4,5,6,7,8,...,90,91,92,93,94,95,96,97,98,99
0,0.0000,-0.004433,0.017344,-0.024087,0.057034,0.068060,0.004883,0.023120,0.083135,-0.049377,...,0.081655,0.047175,-0.026511,0.045658,0.144192,0.069732,-0.036535,0.006796,-0.018133,0.023581
1,-0.9060,0.112829,0.040362,0.090561,0.066377,0.096477,0.076231,0.011750,0.020260,-0.065060,...,0.086467,0.086690,-0.062202,0.067637,0.106305,0.069198,-0.118150,0.094560,0.096149,-0.108009
2,0.6249,0.054528,-0.023325,-0.048565,0.045793,0.095205,-0.129214,0.059908,0.066395,-0.037059,...,0.062776,0.011685,-0.104267,-0.036271,0.054181,0.002265,-0.029882,-0.043883,-0.044142,0.009091
3,-0.4767,-0.047993,0.027005,0.015072,0.136871,-0.029921,0.061309,-0.032258,0.060657,-0.055493,...,0.022087,0.004780,-0.042781,-0.044381,-0.050044,-0.000620,-0.005393,0.052964,0.040219,-0.031443
4,0.0000,0.011156,0.027657,0.059516,0.025263,0.004442,-0.003001,-0.026982,0.040928,0.077745,...,0.043202,0.100658,-0.077168,-0.021634,-0.001215,0.135793,0.031642,-0.031687,0.012914,0.009245


In [4]:
# change compound from continous to categorical variable
own_data["compound"] = np.where(own_data["compound"] > 0, 4, own_data["compound"])      # 4 means positive
own_data["compound"] = np.where(own_data["compound"] == 0, 2, own_data["compound"])     # 2 means neutral
own_data["compound"] = np.where(own_data["compound"] < 0, 0, own_data["compound"])      # 0 means negative
own_data.head()

,compound,0,1,2,3,4,5,6,7,8,...,90,91,92,93,94,95,96,97,98,99
0,2.0,-0.004433,0.017344,-0.024087,0.057034,0.068060,0.004883,0.023120,0.083135,-0.049377,...,0.081655,0.047175,-0.026511,0.045658,0.144192,0.069732,-0.036535,0.006796,-0.018133,0.023581
1,0.0,0.112829,0.040362,0.090561,0.066377,0.096477,0.076231,0.011750,0.020260,-0.065060,...,0.086467,0.086690,-0.062202,0.067637,0.106305,0.069198,-0.118150,0.094560,0.096149,-0.108009
2,4.0,0.054528,-0.023325,-0.048565,0.045793,0.095205,-0.129214,0.059908,0.066395,-0.037059,...,0.062776,0.011685,-0.104267,-0.036271,0.054181,0.002265,-0.029882,-0.043883,-0.044142,0.009091
3,0.0,-0.047993,0.027005,0.015072,0.136871,-0.029921,0.061309,-0.032258,0.060657,-0.055493,...,0.022087,0.004780,-0.042781,-0.044381,-0.050044,-0.000620,-0.005393,0.052964,0.040219,-0.031443
4,2.0,0.011156,0.027657,0.059516,0.025263,0.004442,-0.003001,-0.026982,0.040928,0.077745,...,0.043202,0.100658,-0.077168,-0.021634,-0.001215,0.135793,0.031642,-0.031687,0.012914,0.009245


In [5]:
ext_data = pickle.load(open('data_n_models/eng_ext_twt_doc2vec100.pkl', 'rb'))
ext_data.head()

,target,0,1,2,3,4,5,6,7,8,...,90,91,92,93,94,95,96,97,98,99
0,0,-0.130482,-0.004077,0.015810,0.049959,-0.041490,-0.120463,0.018499,0.054668,0.034647,...,0.042085,0.007140,-0.006839,0.065315,-0.003211,-0.010616,0.007900,-0.021010,0.001334,0.011794
1,0,-0.089091,-0.044952,0.031169,0.031441,0.085560,-0.029774,0.007483,-0.181819,0.014704,...,0.023001,-0.001031,0.057009,0.125231,-0.058873,-0.190622,-0.068788,-0.087910,-0.060429,-0.148368
2,0,-0.034209,-0.032731,-0.006503,-0.015595,0.000750,0.142148,0.008335,0.034191,0.018336,...,-0.003101,-0.033793,-0.046614,-0.036087,0.130222,0.097160,-0.004621,-0.135355,0.075437,0.033472
3,0,0.039511,0.021044,-0.044086,0.040040,0.004851,-0.081538,0.034979,0.090656,-0.017450,...,0.050578,0.034360,-0.055606,0.026050,0.032533,0.052828,0.007092,0.007348,-0.027534,0.068420
4,0,-0.013230,0.019487,-0.037168,0.025121,0.076826,-0.004086,0.005005,0.070915,-0.069859,...,0.088734,0.054730,-0.041834,0.046066,0.089995,0.047997,-0.088571,-0.044634,-0.000533,0.022832


In [8]:
ext_data["target"].value_counts()

0    800000
4    800000
Name: target, dtype: int64

In [9]:
# define X and y variab
X_ext = ext_data.drop("target", axis=1)
y_ext = ext_data["target"]

In [10]:
# split  to test and train set - use 80% for training and 20% for test
X_extrain, X_extest, y_extrain, y_extest = train_test_split(X_ext, y_ext, test_size= 0.2, random_state=42, stratify= y_ext)

In [13]:
clfR = RandomForestClassifier()
clfR.fit(X_extrain, y_extrain)
y_expred = clfR.predict(X_extest)
print(classification_report(y_extest, y_expred))

              precision    recall  f1-score   support

           0       0.67      0.69      0.68    160000
           4       0.68      0.66      0.67    160000

    accuracy                           0.68    320000
   macro avg       0.68      0.68      0.68    320000
weighted avg       0.68      0.68      0.68    320000



In [15]:
X_own = own_data.drop("compound", axis=1)
y_own = own_data["compound"]

In [16]:
y_ownpred = clfR.predict(X_own)
print(classification_report(y_own, y_ownpred))

C:\Users\chiso\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\chiso\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\chiso\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
              precision    recall  f1-score   support

